# Untitled Notebook

This is an initial placeholder notebook. Feel free to edit and rename as well as create your own notebooks, to use Google Cloud Datalab.

In [1]:
# Code cell ready to be run...
print 'Hello!'

Hello!


In [2]:
import gcp.bigquery as bq
import pandas as pd
import numpy as np

In [3]:
%%bigquery schema --table "nyc-tlc:green.trips_2015"

In [4]:
%%sql --module taxiquery
SELECT daynumber, COUNT(*) AS numtrips FROM
    (SELECT DAYOFYEAR(pickup_datetime) AS daynumber FROM [nyc-tlc:green.trips_$YEAR])
GROUP BY daynumber ORDER BY daynumber

In [5]:
trips = bq.Query(taxiquery, YEAR=2015).to_dataframe()
trips[:5]

,daynumber,numtrips
0,1,62943
1,2,43410
2,3,53866
3,4,41602
4,5,41923


In [6]:
avg = np.mean(trips['numtrips'])
print 'Just using average={0} has RMSE of {1}'.format(avg, np.sqrt(np.mean((trips['numtrips'] - avg)**2)))

Just using average=54674.0994475 has RMSE of 10163.4654442


In [7]:
%%sql
SELECT * FROM [fh-bigquery:weather_gsod.stations]
WHERE state = 'NY' AND wban != '99999' AND name contains 'LA GUARDIA'

usaf,wban,name,country,fips,state,call,lat,lon,elev,begin,end
725030,14732,NEW YORK/LA GUARDIA,US,US,NY,KLGA,40779.0,-73880.0,94,19350222,20120619


In [8]:
%%sql --module wxquery
SELECT DAYOFYEAR(TIMESTAMP('$YEAR'+mo+da)) daynumber,
       FIRST(DAYOFWEEK(TIMESTAMP('$YEAR'+mo+da))) dayofweek,
       MIN(min) mintemp, MAX(max) maxtemp, MAX(IF(prcp=99.99,0,prcp)) rain
FROM [fh-bigquery:weather_gsod.gsod$YEAR]
WHERE stn='725030' GROUP BY 1 ORDER BY daynumber DESC